
# Product Recommendation Candidate Generation — Notebook Documentation

This notebook builds a comprehensive set of candidate products for personalized recommendations using multiple strategies:

1. **Active Users Identification**  
   - Extracts distinct active users from recent product interactions.

2. **User History**  
   - Recommends products previously interacted with by the user.

3. **Same Category**  
   - Suggests products from categories the user prefers.

4. **Brand Affinity**  
   - Recommends products from brands the user has purchased before.

5. **Frequently Bought Together (FBT)**  
   - Suggests products commonly bought together with those the user interacted with.

6. **Age Group Popular**  
   - Recommends products popular among users in the same age group.

7. **Trending Products**  
   - Suggests top 50 trending products from the last 30 days.

8. **Discount Based**  
   - Recommends top 50 products with the highest discounts.

9. **Location Popularity**  
   - Suggests products popular in the user's location.

10. **Union and Deduplication**  
    - Combines all candidate sources, removes duplicates, and limits to 200 products per user.

11. **Output**  
    - Displays sample candidates and saves the final candidate set to a Delta table for downstream use.

---
**Table Saved:**  
`kusha_solutions.product_recomendation.gold_candidate_products`

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark.conf.set("spark.databricks.remoteFiltering.blockSelfJoins", "false")
spark.conf.set("spark.sql.shuffle.partitions", "200")

gold = "kusha_solutions.product_recomendation"

# --------------------------------------------------
# ACTIVE USERS (ONLINE CONTEXT)
# --------------------------------------------------
active_users = (
    spark.table(f"{gold}.gold_user_product_interactions")
         .select("CustomerID")
         .distinct()
)

# --------------------------------------------------
# 1️⃣ USER HISTORY
# --------------------------------------------------
user_history = (
    spark.table(f"{gold}.gold_user_product_interactions")
         .join(active_users, "CustomerID")
         .select("CustomerID", "ProductID")
         .withColumn("candidate_source", F.lit("user_history"))
)

# --------------------------------------------------
# 2️⃣ SAME CATEGORY
# --------------------------------------------------
same_category = (
    spark.table(f"{gold}.gold_user_category_preferences")
         .join(active_users, "CustomerID")
         .join(
             spark.table(f"{gold}.gold_product_features"),
             "Category"
         )
         .select("CustomerID", "ProductID")
         .withColumn("candidate_source", F.lit("same_category"))
)

# --------------------------------------------------
# 3️⃣ BRAND AFFINITY
# --------------------------------------------------
brand_affinity = (
    spark.table(f"{gold}.gold_sales_enriched")
         .select("CustomerID", "Brand")
         .distinct()
         .join(active_users, "CustomerID")
         .join(
             spark.table(f"{gold}.gold_product_features"),
             "Brand"
         )
         .select("CustomerID", "ProductID")
         .withColumn("candidate_source", F.lit("brand_affinity"))
)

# --------------------------------------------------
# 4️⃣ FREQUENTLY BOUGHT TOGETHER (FBT)
# --------------------------------------------------
fbt = (
    spark.table(f"{gold}.gold_user_product_interactions")
         .join(active_users, "CustomerID")
         .join(
             spark.table(f"{gold}.gold_frequently_bought_together"),
             F.col("ProductID") == F.col("ProductID_A"),
             "inner"
         )
         .select(
             "CustomerID",
             F.col("ProductID_B").alias("ProductID")
         )
         .withColumn("candidate_source", F.lit("fbt"))
)

# --------------------------------------------------
# 5️⃣ AGE GROUP POPULAR
# --------------------------------------------------
age_group = (
    spark.table(f"{gold}.gold_customers_with_age_group")
         .join(active_users, "CustomerID")
         .join(
             spark.table(f"{gold}.gold_agegroup_popular_products"),
             "AgeGroup"
         )
         .select("CustomerID", "ProductID")
         .withColumn("candidate_source", F.lit("age_group"))
)

# --------------------------------------------------
# 6️⃣ TRENDING (TOP 50 ONLY — SAFE)
# --------------------------------------------------
trending = (
    active_users
      .crossJoin(
          spark.table(f"{gold}.gold_trending_products_30d")
               .orderBy(F.desc("purchase_count"))
               .limit(50)
               .select("ProductID")
      )
      .withColumn("candidate_source", F.lit("trending"))
)

# --------------------------------------------------
# 7️⃣ DISCOUNT BASED
# --------------------------------------------------
discount = (
    active_users
      .crossJoin(
          spark.table(f"{gold}.gold_product_features")
               .filter("DiscountPercent > 0")
               .orderBy(F.desc("DiscountPercent"))
               .limit(50)
               .select("ProductID")
      )
      .withColumn("candidate_source", F.lit("discount"))
)

# --------------------------------------------------
# 8️⃣ LOCATION POPULARITY
# --------------------------------------------------
location = (
    spark.table(f"{gold}.gold_sales_enriched")
         .groupBy("CustomerLocation", "ProductID")
         .count()
         .join(
             spark.table(f"{gold}.gold_customers_with_age_group")
                  .select(
                      "CustomerID",
                      F.col("Location").alias("CustomerLocation")
                  )
                  .join(active_users, "CustomerID"),
             "CustomerLocation"
         )
         .select("CustomerID", "ProductID")
         .withColumn("candidate_source", F.lit("location"))
)

# --------------------------------------------------
# 🔗 UNION ALL
# --------------------------------------------------
all_candidates = (
    user_history
      .unionByName(same_category)
      .unionByName(brand_affinity)
      .unionByName(fbt)
      .unionByName(age_group)
      .unionByName(trending)
      .unionByName(discount)
      .unionByName(location)
)

# --------------------------------------------------
# 🎯 DEDUP + LIMIT PER USER (ONLINE SAFE)
# --------------------------------------------------
w = Window.partitionBy("CustomerID").orderBy("ProductID")

final_candidates = (
    all_candidates
      .dropDuplicates(["CustomerID", "ProductID"])
      .withColumn("rn", F.row_number().over(w))
      .filter("rn <= 200")
      .drop("rn")
)

display(final_candidates.limit(100))


In [0]:
final_candidates.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable(f"{gold}.gold_candidate_products")
